# Accenture Supply Chain Challenge

In [ ]:
import pandas as pd
import numpy as np

data_dir = './data/'

## Global Variables

In [ ]:
# ports that are the same but have different names (data cleaning)
same_ports =  {
    'ATHENAS': 'Athens',
    'BCN': 'Barcelona',
}

# converters function for pd.read_csv()
convs = {
    'origin_port': lambda x: same_ports[x] if x in same_ports else x
}

## Orders

In [ ]:
orders = pd.read_csv(data_dir + "orders.csv", sep=';', converters=convs, na_filter=False)

print(f'Total number of orders: {orders.size}')
orders.head()

In [ ]:
# orders grouped by origin port
orders_per_port = orders.groupby(by='origin_port').agg({'order_id': 'count', 'units': 'sum'})
orders_per_port

In [ ]:
# orders grouped by origin port and logistic hub
orders.groupby(by=['origin_port','logistic_hub']).agg({'units': 'sum'})

### Effect of Customs Procedures on Lateness

In [ ]:
a = orders[orders['late_order'] == True].groupby(by=['origin_port', 'customs_procedures']).count()[['order_id']]
b = orders.groupby(by=['origin_port', 'customs_procedures']).count()[['order_id']]
c = a/b

c.plot(kind='bar')

### Effect of Origin Port on Lateness

In [ ]:
a = orders[orders['late_order'] == True].groupby(by=['origin_port']).count()[['order_id']]
b = orders.groupby(by=['origin_port']).count()[['order_id']]
c = a/b

c.plot(kind='bar')

### Effect of Logistic Hub on Lateness

In [ ]:
a = orders[orders['late_order'] == True].groupby(by=['logistic_hub']).count()[['order_id']]
b = orders.groupby(by=['logistic_hub']).count()[['order_id']]
c = (a/b).sort_values(by='order_id')

c.plot(kind='bar')

### Effect of 3rd-party Logistics on Lateness

In [ ]:
a = orders[orders['late_order'] == True].groupby(by=['3pl']).count()[['order_id']]
b = orders.groupby(by=['3pl']).count()[['order_id']]
c = a/b

c.plot(kind='bar')

### Effect of Number of Units on Lateness

In [ ]:
max_units = max(orders['units'])
unit_groups = 7
unit_steps = np.linspace(0, max_units, unit_groups + 1)
print(f'max_units: {max_units}')
print(f'unit_groups: {unit_groups}')
print(f'unit_steps: {unit_steps}')

In [ ]:
late_percentages = []
for i in range(len(unit_steps) - 1):
    x = orders[orders['units'].between(unit_steps[i], unit_steps[i+1])].groupby('late_order').count()[['units']]
    num_late = x['units'][True]
    num_in_range = x['units'][True] + x['units'][False]
    late_percentages.append(num_late / num_in_range)
    print(f'[{unit_steps[i]},{unit_steps[i+1]}]: num_late={num_late}, num_in_range={num_in_range}, percentage={late_percentages[i]}')

#a = orders[orders['late_order'] == True].groupby(by=['units']).count()[['order_id']]
#b = orders.groupby(by=['units']).count()[['order_id']]
#c = a/b

pd.Series(late_percentages, unit_steps[:-1]).plot()

#c.plot(kind='bar')
#orders[orders['units'] < 500].sort_values(by='units').groupby('late_order').sum()[['units']]

## Cities

In [ ]:
cities_data = pd.read_csv(data_dir + 'cities_data.csv', sep=';')

cities_data.head()

## Cities with Costs

In [ ]:
cities_costs = pd.read_csv(data_dir + 'cities_data_costs.csv', sep=',')

cities_costs.head()

## Product Attributes

In [ ]:
product_attributes = pd.read_csv(data_dir + 'product_attributes.csv', sep=',')

product_attributes.head()

In [ ]:
# product attributes grouped by material handling
product_attributes.groupby(by='material_handling').agg({'product_id': 'count', 'weight': 'sum'})

## Product Weight Classes

In [ ]:
product_weight_classes = pd.read_csv(data_dir + 'product_weight_class.csv', sep=',')

product_weight_classes.head()

In [ ]:
# grouped by weight classes
product_weight_classes.groupby(by='weight_class').agg({'product_id': 'count'})